In [1]:
import ee
ee.Initialize()

In [2]:
from iteru import *

In [3]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [4]:
# startYear = 2020
# startMonth =1
# startDay = 1
# endYear = 2022
# endMonth = 3
# endDay = 12

In [5]:
# SAR_col= S1_SAR_col(GERD_aoi,startYear,startMonth,startDay,endYear,endMonth,endDay,temp_freq=None)

In [6]:
# SAR=SAR_col[0];dates_sequences = SAR_col[1]

In [7]:
# url = SAR_timeseries_url(SAR,GERD_aoi,vis_method='rgb')

In [8]:
# out_gif = get_gif(url)
# add_text_to_gif(out_gif,dates_sequences, dates_font_size = 25, dates_font_color = 'white', framesPerSecond = 2)
# display_gif(out_gif)

In [9]:
# water_vectors = SAR.map(filterSpeckles).map(water_classify).map(water_to_vector)

In [10]:
#water_area = [area /(1e6) for area in water_vectors.aggregate_array('Area').getInfo()]

In [11]:
# dems = water_vectors.map(max_water_ele)

In [12]:
# max_elev = [elev for elev in dems.aggregate_array('Maximum_water_elevation').getInfo()]

In [13]:
# volume_stats = dems.map(water_vol)

In [14]:
# ele_sum = volume_stats.aggregate_array('Elevation_sum').getInfo()

In [15]:
# pixel_num  = volume_stats.aggregate_array('Pixels_number').getInfo()

In [16]:
# volume = [((water_level*pixles_count-elevations_sum)*900)/(1e9) for water_level,elevations_sum , pixles_count in zip(max_elev,ele_sum,pixel_num)]

In [17]:
# GERD_SAR_timelaspe(  aoi = GERD_aoi,
#                      startYear = 2020,
#                      startMonth = 1,
#                      startDay = 1,
#                      endYear = 2022,
#                      endMonth = 3,
#                      endDay = 10,
#                      temp_freq=None, 
#                      vis_method = 'single_band_VH',
#                      crs='EPSG:3857',
#                      dimensions=1080,
#                      dates_font_size = 20,
#                      copywrite_font_size = 12,
#                      dates_font_color = 'black',
#                      copywrite_font_color = 'blue',
#                      framesPerSecond = 2,                     
#                     )

In [45]:
# stats_dict = GERD_water_stats(  aoi=GERD_aoi,
#                      startYear=2020,
#                      startMonth=6,
#                      startDay=1,
#                      endYear=2022,
#                      endMonth=3,
#                      endDay=10,
#                      temp_freq='monthly',
#                      water_area=True,
#                      water_level=True,
#                      water_volume=True )

In [47]:
stats_dict['Image_date']

['2020-6-1',
 '2020-7-1',
 '2020-7-31',
 '2020-8-30',
 '2020-9-29',
 '2020-10-29',
 '2020-11-28',
 '2020-12-28',
 '2021-1-27',
 '2021-2-26',
 '2021-3-28',
 '2021-4-27',
 '2021-5-27',
 '2021-6-26',
 '2021-7-26',
 '2021-8-25',
 '2021-9-24',
 '2021-10-24',
 '2021-11-23',
 '2021-12-23',
 '2022-1-22']